In [3]:
import yfinance as yf
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from datetime import datetime, timedelta

# Download data
end = datetime.now()
start = datetime(end.year - 2, end.month, end.day)
stock = "BTC-USD"
bit_coin_data = yf.download(stock, start, end)
Closing_price = bit_coin_data[['Close']]
Open_price = bit_coin_data[['Open']]

# Preprocess data
data = Closing_price.values
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(data)

# Create dataset
def create_dataset(data, time_step=1):
    X, y = [], []
    for i in range(len(data) - time_step - 1):
        a = data[i:(i + time_step), 0]
        X.append(a)
        y.append(data[i + time_step, 0])
    return np.array(X), np.array(y)

time_step = 60  # Number of time steps for LSTM
X, y = create_dataset(scaled_data, time_step)

# Reshape for LSTM
X = X.reshape(X.shape[0], X.shape[1], 1)

# Split into train and test sets
split = int(len(X) * 0.8)
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]

# Build LSTM model
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(time_step, 1)))
model.add(LSTM(units=50))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mean_squared_error')

# Train model
model.fit(X_train, y_train, epochs=20, batch_size=32)

# Make predictions
predictions = model.predict(X_test)
predictions = scaler.inverse_transform(predictions)

# Inverse transform y_test
y_test = scaler.inverse_transform(y_test.reshape(-1, 1))

# Forecast next 15 days
last_sequence = scaled_data[-time_step:]
forecast = []

for _ in range(15):
    last_sequence = last_sequence.reshape((1, time_step, 1))
    next_price = model.predict(last_sequence)
    # Add randomness to forecast
    next_price_with_noise = next_price + np.random.uniform(-0.03, 0.03)
    forecast.append(next_price_with_noise[0, 0])
    # Update last_sequence for the next prediction
    last_sequence = np.roll(last_sequence, -1, axis=1)
    last_sequence[0, -1, 0] = next_price_with_noise

forecast = scaler.inverse_transform(np.array(forecast).reshape(-1, 1))

# Generate dates for forecast
future_dates = [end + timedelta(days=i) for i in range(1, 16)]

# Plotting

# 1. Grouped bar graph of Open and Close prices for previous 15 days
recent_15_days = bit_coin_data.tail(15)
fig = go.Figure()

# Add Open prices
fig.add_trace(go.Bar(x=recent_15_days.index, y=recent_15_days['Open'], name='Open Price', marker_color='red'))

# Add Close prices
fig.add_trace(go.Bar(x=recent_15_days.index, y=recent_15_days['Close'], name='Close Price', marker_color='blue'))

fig.update_layout(title='Open and Close Prices for Last 15 Days',
                  xaxis_title='Date',
                  yaxis_title='Price',
                  barmode='group')

fig.show()

# 2. Close price plot of previous 2 years
fig = go.Figure()

# Add close price
fig.add_trace(go.Scatter(x=Closing_price.index, y=Closing_price['Close'], mode='lines', name='Close Price'))

fig.update_layout(title='Bitcoin Close Price for Last 2 Years',
                  xaxis_title='Date',
                  yaxis_title='Price')

fig.show()

# 3. Actual vs Predicted Prices for Last 15 Days
# Assuming the last 15 days are used for comparison
actual_15_days = Closing_price.tail(15)
predicted_15_days = forecast.flatten()

fig = go.Figure()

# Add Actual Prices
fig.add_trace(go.Bar(x=actual_15_days.index, y=actual_15_days['Close'], name='Actual Prices', marker_color='green'))

# Add Predicted Prices
fig.add_trace(go.Bar(x=actual_15_days.index, y=predicted_15_days, name='Predicted Prices', marker_color='blue'))

fig.update_layout(title='Actual vs Predicted Prices for Last 15 Days',
                  xaxis_title='Date',
                  yaxis_title='Price',
                  barmode='group')

fig.show()

# 4. Future 15 Days Predictions
fig = go.Figure()

# Add historical prices
fig.add_trace(go.Scatter(x=Closing_price.index, y=Closing_price['Close'], mode='lines', name='Historical Prices'))

# Add forecast prices
fig.add_trace(go.Scatter(x=future_dates, y=forecast.flatten(), mode='lines', name='Forecast Prices'))

fig.update_layout(title='Bitcoin Price Forecast for Next 15 Days',
                  xaxis_title='Date',
                  yaxis_title='Price')

fig.show()

model.save('model.h5')


[*********************100%***********************]  1 of 1 completed

Epoch 1/20



/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



17/17 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - loss: 0.0672
Epoch 2/20
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 0.0062
Epoch 3/20
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 0.0023
Epoch 4/20
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 0.0019
Epoch 5/20
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 0.0014
Epoch 6/20
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 0.0016
Epoch 7/20
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 0.0015
Epoch 8/20
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - loss: 0.0012
Epoch 9/20
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 0.0013
Epoch 10/20
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - loss: 0.0017
Epoch 11/20
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - loss: 0.0014
Epoch 12/20
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 0.0013
Epoch 13/20
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 0.0014
Epoch 14/20
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 0.0013
Epoch 15/20
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 0.0013
Epoch 16/20
17/

/var/folders/vn/knr3fr2541516prk5124htt80000gn/T/ipykernel_1479/1803598673.py:72: DeprecationWarning:

Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)



1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step


In [2]:
model.save('model.h5')